In [1]:
import numpy as np
import pickle
import torch
from torch import nn
import collections
from model import DQN_pytorch

Using TensorFlow backend.
/home/synthesisproject/anaconda3/envs/syn_gen_release/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/synthesisproject/anaconda3/envs/syn_gen_release/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/synthesisproject/anaconda3/envs/syn_gen_release/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 =

[array([0., 1., 0., 0., 0.])]
[5, 2]
torch.Size([1, 40, 115])
torch.Size([1, 5])
torch.Size([1, 80])
torch.Size([1, 10])
tensor([[-0.1549]], grad_fn=<AddmmBackward>)
torch.Size([1, 1])


In [2]:
# Load Q_data_random
with open('./data/Q_data_random.pkl', 'rb') as f:
    Q_data_random = pickle.load(f)

print(len(Q_data_random))

1000


In [3]:
dqn = DQN_pytorch()

s_material_inputs = []
s_step_inputs     = []
a_elem_inputs     = []
a_comp_inputs     = []
Q_targets         = []

for episode in Q_data_random:
    for step in episode:
        # state, action, reward = step
        [s_material, s_step], [a_elem, a_comp], Q_target = step

        s_material = torch.tensor(s_material)
        s_step = torch.tensor(s_step)
        a_elem = torch.tensor(a_elem)
        a_comp = torch.tensor(a_comp)
        Q_target = torch.tensor([Q_target])

        s_material_inputs.append(s_material)
        s_step_inputs.append(s_step)
        a_elem_inputs.append(a_elem)
        a_comp_inputs.append(a_comp)
        Q_targets.append(Q_target)

s_material_inputs = torch.stack(s_material_inputs)
s_step_inputs = torch.stack(s_step_inputs)
a_elem_inputs = torch.stack(a_elem_inputs)
a_comp_inputs = torch.stack(a_comp_inputs)
Q_targets = torch.stack(Q_targets)

print('s_material_inputs:', s_material_inputs.shape)
print('s_step_inputs:', s_step_inputs.shape)
print('a_elem_inputs:', a_elem_inputs.shape)
print('a_comp_inputs:', a_comp_inputs.shape)
print('Q_targets:', Q_targets.shape)
print('')

s_material_inputs: torch.Size([5000, 40, 115])
s_step_inputs: torch.Size([5000, 5])
a_elem_inputs: torch.Size([5000, 80])
a_comp_inputs: torch.Size([5000, 10])
Q_targets: torch.Size([5000, 1])



In [4]:
# Train Q network
optimizer = torch.optim.Adam(dqn.parameters(), lr=1e-2)
loss_func = torch.nn.SmoothL1Loss().float()
for epoch in range(100):
    prediction = dqn(s_material = s_material_inputs, # torch.Size([batch_size, 40, 115]) and must be .float() !!!
                    s_step      = s_step_inputs,     # torch.Size([batch_size,5])
                    a_elem      = a_elem_inputs,     # torch.Size([batch_size,80])
                    a_comp      = a_comp_inputs,  # torch.Size([batch_size,10])
                    )                    # Input x and predict based on x
    loss       = loss_func(prediction.float(), Q_targets.float()).float()   # Must be (1. nn output, 2. target)
    optimizer.zero_grad()   # Clear gradients for next train
    loss.backward()         # Backpropagation, compute gradients
    optimizer.step()        # Apply gradients
    print('Epoch = ', epoch, 'Loss = %.4f' % loss.data.numpy())

Epoch =  0 Loss = 864.4478
Epoch =  1 Loss = 864.3575
Epoch =  2 Loss = 864.2665
Epoch =  3 Loss = 864.1745
Epoch =  4 Loss = 864.0809
Epoch =  5 Loss = 863.9853
Epoch =  6 Loss = 863.8869
Epoch =  7 Loss = 863.7855
Epoch =  8 Loss = 863.6807
Epoch =  9 Loss = 863.5716
Epoch =  10 Loss = 863.4580
Epoch =  11 Loss = 863.3397
Epoch =  12 Loss = 863.2162
Epoch =  13 Loss = 863.0870
Epoch =  14 Loss = 862.9518
Epoch =  15 Loss = 862.8105
Epoch =  16 Loss = 862.6624
Epoch =  17 Loss = 862.5072
Epoch =  18 Loss = 862.3446
Epoch =  19 Loss = 862.1741
Epoch =  20 Loss = 861.9954
Epoch =  21 Loss = 861.8081
Epoch =  22 Loss = 861.6115
Epoch =  23 Loss = 861.4056
Epoch =  24 Loss = 861.1898
Epoch =  25 Loss = 860.9636
Epoch =  26 Loss = 860.7266
Epoch =  27 Loss = 860.4785
Epoch =  28 Loss = 860.2184
Epoch =  29 Loss = 859.9463
Epoch =  30 Loss = 859.6613
Epoch =  31 Loss = 859.3631
Epoch =  32 Loss = 859.0514
Epoch =  33 Loss = 858.7251
Epoch =  34 Loss = 858.3842
Epoch =  35 Loss = 858.0276
Ep